# 03. 动态类型（Dynamic Typing）

Python 的类型属于对象而不是变量名。本节把“名字绑定/引用共享/原地修改 vs 重新绑定/可变默认参数坑”讲透。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 01 节：可变/不可变


## 知识点地图

- 1. 名字绑定：a=b 不是复制值
- 2. 原地修改 vs 重新绑定：append vs +
- 3. 强类型：需要显式转换
- 4. 传参：函数拿到的是对象引用
- 5. 避免副作用：先复制再改（纯函数风格）
- 6. 可变默认参数坑：默认值只求一次
- 7. 调试：type/id/repr 三板斧


## 自检清单（学完打勾）

- [ ] 理解变量名只是引用（标签）
- [ ] 能用 id() 证明 append 与 + 的差异
- [ ] 理解传参是传对象引用，修改可变对象会影响外部
- [ ] 掌握可变默认参数坑与 None 修复法
- [ ] 知道什么时候需要复制（list(x)/x.copy()）


## 知识点 1：名字绑定：a=b 不是复制值

变量名指向对象；a=b 让两个名字指向同一对象。


In [1]:
a = [1, 2]
b = a
b.append(3)
print(a, b, a is b)


[1, 2, 3] [1, 2, 3] True


## 知识点 2：原地修改 vs 重新绑定：append vs +

- append/extend：原地修改（对象 id 不变）
- b=b+[x]：创建新列表并重新绑定 b（b 的 id 变）



In [2]:
a = [1, 2]
b = a
print('id(a)=', id(a), 'id(b)=', id(b))

b.append(3)
print('after append:', a, b, id(a), id(b))

b = b + [4]
print('after +:', a, b)
print('id(a)=', id(a), 'id(b)=', id(b))


id(a)= 2127709504000 id(b)= 2127709504000
after append: [1, 2, 3] [1, 2, 3] 2127709504000 2127709504000
after +: [1, 2, 3] [1, 2, 3, 4]
id(a)= 2127709504000 id(b)= 2127708263168


## 知识点 3：强类型：需要显式转换

Python 不会把 '1' 自动当作 1；需要 int('1')。


In [3]:
print(int('1') + 2)


3


## 知识点 4：传参：函数拿到的是对象引用

函数参数与外部变量名可能引用同一对象。
如果函数内原地修改可变对象，外部也会看到变化。



In [4]:
def add_item(lst, x):
    lst.append(x)

nums = [1, 2]
add_item(nums, 3)
print(nums)


[1, 2, 3]


## 知识点 5：避免副作用：先复制再改（纯函数风格）

当你不希望修改调用者传入的列表时，用 list(x)/copy() 复制后再修改。


In [5]:
def add_item_pure(lst, x):
    new_lst = list(lst)
    new_lst.append(x)
    return new_lst

nums = [1, 2]
print(add_item_pure(nums, 3), nums)


[1, 2, 3] [1, 2]


## 知识点 6：可变默认参数坑：默认值只求一次

默认参数在函数定义时只求值一次。
因此 buf=[] 会在多次调用间复用同一个对象，导致状态泄漏。



In [6]:
def bad(x, buf=[]):
    buf.append(x)
    return buf

print(bad(1))
print(bad(2))


def good(x, buf=None):
    if buf is None:
        buf = []
    buf.append(x)
    return buf

print(good(1))
print(good(2))


[1]
[1, 2]
[1]
[2]


## 知识点 7：调试：type/id/repr 三板斧

遇到‘怎么变了’优先打印类型、id 与 repr，定位是“同一对象”还是“新对象”。


In [7]:
x = {'a': [1, 2]}
print(type(x), id(x), repr(x))


<class 'dict'> 2127709409472 {'a': [1, 2]}


## 常见坑

- 可变对象被多个变量引用时，修改会联动
- 不要用可变对象做默认参数


## 综合小案例：实现 push：可选原地修改

实现 push(stack, x, *, inplace=False)：
- inplace=True：原地 append 并返回同一对象
- inplace=False：返回新列表，不修改 stack



In [ ]:
def push(stack, x, *, inplace=False):
    if inplace:
        stack.append(x)
        return stack
    new_stack = list(stack)
    new_stack.append(x)
    return new_stack

s = [1, 2]
print(push(s, 3), s)
print(push(s, 4, inplace=True), s)


## 自测题（不写代码也能回答）

- 为什么说 Python 是动态类型但强类型？
- append 与 + 的差别是什么？
- 可变默认参数问题的根因是什么？


## 练习题（建议写代码）

- 写 safe_extend(lst, items)：返回新列表，不修改 lst。
- 写一个例子证明：函数内修改 dict 会影响外部调用者。
